In [ ]:
!pip install --quiet apache_beam

     |████████████████████████████████| 8.3MB 2.2MB/s 
     |████████████████████████████████| 829kB 44.5MB/s 
     |████████████████████████████████| 63.8MB 125kB/s 
     |████████████████████████████████| 81kB 10.0MB/s 
     |████████████████████████████████| 51kB 6.5MB/s 
     |████████████████████████████████| 1.4MB 47.9MB/s 
     |████████████████████████████████| 153kB 54.7MB/s 
     |████████████████████████████████| 61kB 7.3MB/s 
     |████████████████████████████████| 112kB 46.1MB/s 
ERROR: pydrive 1.3.1 has requirement oauth2client>=4.0.0, but you'll have oauth2client 3.0.0 which is incompatible.
ERROR: multiprocess 0.70.10 has requirement dill>=0.3.2, but you'll have dill 0.3.1.1 which is incompatible.


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving data.txt to data.txt


In [ ]:
!ls

dept_data.txt  sample_data


In [ ]:
import apache_beam as beam

def SplitRow(element):
    return element.split(',')

def filtering(record):
  return record[3] == 'Accounts'


p1 = beam.Pipeline()

attendance_count = (
    
   p1
    |beam.io.ReadFromText('dept_data.txt')
    |beam.Map(SplitRow)
   #| beam.Map(lambda record: record.split(','))

    |beam.Filter(filtering)
  # |beam.Filter(lambda record: record[3] == 'Accounts')
    
    |beam.Map(lambda record: (record[1], 1))
    |beam.CombinePerKey(sum)
    
    |beam.io.WriteToText('data/output_new_final')
  
)

p1.run()

In [ ]:
!head -n 20 data/output_new_final-00000-of-00001

('Marco', 31)
('Rebekah', 31)
('Itoe', 31)
('Edouard', 31)
('Kyle', 62)
('Kumiko', 31)
('Gaston', 31)
('Ayumi', 30)


In [ ]:
import apache_beam as beam

def SplitRow(element):
    return element.split(',')

p = beam.Pipeline()


input_collection = ( 
                      p 
                      | "Read from text file" >> beam.io.ReadFromText('dept_data.txt')
                      | "Split rows" >> beam.Map(SplitRow)
                   )

accounts_count = (
                      input_collection
                      | 'Get all Accounts dept persons' >> beam.Filter(lambda record: record[3] == 'Accounts')
                      | 'Pair each accounts employee with 1' >> beam.Map(lambda record: ("Accounts, " +record[1], 1))
                      | 'Group and sum1' >> beam.CombinePerKey(sum)
                    #  | 'Write results for account' >> beam.io.WriteToText('data/Account')
                 )

hr_count = (
                input_collection
                | 'Get all HR dept persons' >> beam.Filter(lambda record: record[3] == 'HR')
                | 'Pair each hr employee with 1' >> beam.Map(lambda record: ("HR, " +record[1], 1))
                | 'Group and sum' >> beam.CombinePerKey(sum)
                #| 'Write results for hr' >> beam.io.WriteToText('data/HR')
           )

output =(
         (accounts_count,hr_count)
    | beam.Flatten()
    | beam.io.WriteToText('data/both')
)



p.run()
  
# Sample the first 20 results, remember there are no ordering guarantees.
!{('head -n 20 data/both-00000-of-00001')}


#!{('head -n 20 data/HR-00000-of-00001')}

In [ ]:
!head -n 20 data.txt

	KING LEAR


	DRAMATIS PERSONAE


LEAR	king of Britain  (KING LEAR:)

KING OF FRANCE:

DUKE OF BURGUNDY	(BURGUNDY:)

DUKE OF CORNWALL	(CORNWALL:)

DUKE OF ALBANY	(ALBANY:)

EARL OF KENT	(KENT:)

EARL OF GLOUCESTER	(GLOUCESTER:)



In [ ]:
from past.builtins import unicode
import re

import apache_beam as beam
from apache_beam.io import ReadFromText
from apache_beam.io import WriteToText
from apache_beam.options.pipeline_options import PipelineOptions
from apache_beam.options.pipeline_options import SetupOptions


with beam.Pipeline(options=PipelineOptions()) as p:

  # Read the text file[pattern] into a PCollection.
  lines = p | ReadFromText('data.txt')

   # Format the counts into a PCollection of strings.
  def format_result(word_count):
    (word, count) = word_count
    return '%s: %s' % (word, count)

  # Count the occurrences of each word.
  counts = (
      lines
      | 'Split' >> (
          beam.FlatMap(lambda x: re.findall(r'[A-Za-z\']+', x)).
          with_output_types(unicode))
      | 'PairWithOne' >> beam.Map(lambda x: (x, 1))
      | 'GroupAndSum' >> beam.CombinePerKey(sum)
      | 'Format' >> beam.Map(format_result)
      |beam.io.WriteToText('data/output_new_final'))


In [ ]:
!{('head -n 20 data/output_new_final-00000-of-00001')}

KING: 243
LEAR: 236
DRAMATIS: 1
PERSONAE: 1
king: 65
of: 447
Britain: 2
OF: 15
FRANCE: 10
DUKE: 3
BURGUNDY: 8
CORNWALL: 63
ALBANY: 67
EARL: 2
KENT: 156
GLOUCESTER: 141
EDGAR: 126
son: 29
to: 438
Gloucester: 26
